In [3]:
import pandas as pd
import numpy as np
import requests

In [22]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
header = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest"
}

req = requests.get(url,headers=header)
tab = pd.read_html(req.text)
df_tor = pd.DataFrame(tab[0])

df_tor.columns = ['PostalCode','Borough','Neighborhood']

df_tor.reset_index(drop=True,inplace = True)
df_tor.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [23]:
df_assigned = df_tor[df_tor.Borough != 'Not assigned'].reset_index(drop=True)
df_assigned.head()
#df_final  =df_tor.groupby("PostalCode").agg(lambda x:','.join(set(x)))

#df_final.loc[df_final['Neighborhood']=="Not assigned",'Neighborhood']=df_final.loc[df_final['Neighborhood']=="Not assigned",'Borough']


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [24]:
df_group = df_assigned.groupby(['PostalCode','Borough'], as_index=False).agg(lambda x: ','.join(x))
df_group.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [27]:
not_assign = df_group.Neighborhood == 'Not assigned'
df_group.loc[not_assign, 'Neighborhood'] = df_group.loc[not_assign, 'Borough']

In [28]:
df_tor_cleaned = df_group
df_tor_cleaned.shape

(103, 3)

## The second part of the project begins here
### GEOCODER PACKAGE


In [7]:
!wget -q -O "toronto.csv" http://cocl.us/Geospatial_data 

In [8]:
geo_cor = pd.read_csv('toronto.csv')

In [29]:
geo_cor.shape
geo_cor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [33]:
df_tor_index = df_tor_cleaned.set_index('PostalCode')
geo_tmp = geo_cor.set_index('Postal Code')
df_toronto_comb = pd.concat([df_tor_index,geo_tmp],axis=1,join='inner')

df_toronto_comb.index.name = 'PostalCode'
df_toronto_comb.reset_index(inplace=True)

df_toronto_comb.shape

(103, 5)

In [35]:
df_toronto_comb

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Scarborough,43.806686,-79.194353
1,M1C,Scarborough,Scarborough,43.784535,-79.160497
2,M1E,Scarborough,Scarborough,43.763573,-79.188711
3,M1G,Scarborough,Scarborough,43.770992,-79.216917
4,M1H,Scarborough,Scarborough,43.773136,-79.239476
5,M1J,Scarborough,Scarborough,43.744734,-79.239476
6,M1K,Scarborough,Scarborough,43.727929,-79.262029
7,M1L,Scarborough,Scarborough,43.711112,-79.284577
8,M1M,Scarborough,Scarborough,43.716316,-79.239476
9,M1N,Scarborough,Scarborough,43.692657,-79.264848
